# Pharmacy Claims ETL – Python & SQL Comparison
This notebook simulates a Databricks-style ETL pipeline using Python (Pandas) and shows the equivalent SQL logic.

In [ ]:
import pandas as pd

## 🔽 Load Synthetic Claims Data

In [ ]:
df = pd.read_csv('claims_raw.csv', parse_dates=['claim_date'])
df.head()

## ⚙️ Python ETL Logic (Simulating Databricks Job)

In [ ]:
# Filter paid claims
df_paid = df[df['claim_status'] == 'Paid'].copy()

# Create derived columns
df_paid['claim_month'] = df_paid['claim_date'].dt.to_period('M').astype(str)
df_paid['high_cost_flag'] = df_paid['claim_cost'] > 100

# Group and summarize
summary = df_paid.groupby(['claim_month', 'drug_code']).agg(
    total_claims=('claim_id', 'count'),
    total_cost=('claim_cost', 'sum'),
    avg_cost=('claim_cost', 'mean'),
    high_cost_count=('high_cost_flag', 'sum')
).reset_index()
summary

## 🧾 Equivalent SQL Logic

```sql
-- Step 1: Filter to only paid claims
WITH paid_claims AS (
    SELECT *
    FROM claims_raw
    WHERE claim_status = 'Paid'
),

-- Step 2: Add derived columns (claim_month, high_cost_flag)
derived_claims AS (
    SELECT *,
           FORMAT(claim_date, 'yyyy-MM') AS claim_month,
           CASE WHEN claim_cost > 100 THEN 1 ELSE 0 END AS high_cost_flag
    FROM paid_claims
)

-- Step 3: Aggregate summary
SELECT 
    claim_month,
    drug_code,
    COUNT(claim_id) AS total_claims,
    SUM(claim_cost) AS total_cost,
    AVG(claim_cost) AS avg_cost,
    SUM(high_cost_flag) AS high_cost_count
FROM derived_claims
GROUP BY claim_month, drug_code
ORDER BY claim_month, drug_code;
```